In [6]:
#import libraries and such
from collections import deque
from random import random
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import requests
from scipy.stats import expon
import seaborn as sns

In [ ]:
## global variables
sns.set()
N = 20
green_time = 15.
cycle = 30.
trip_length = 10
saturation =  .95
arrival_rate = (saturation * green_time)/(trip_length*cycle) #am i calculating this right?
interval_generator = expon(scale=1./arrival_rate)
print(arrival_rate)

# classes
class Queue(deque):
    #static properties
    population = 0
    
    #initializing a queue, inherits from deque class
    def __init__(self, iterable=(), maxlen=None):
        deque.__init__(self, iterable,maxlen)
        self.id = Queue.population
        self.history = []
        Queue.population+=1
        self.nextTime = interval_generator.rvs(1)
        
    #record the queue length at a moment in time
    def record(self, time):
        self.history.append([time,len(self)])
    
    def tick(self,time):
        if self.nextTime <= time:
            self.appendleft(Car(time))
            self.nextTime = time+interval_generator.rvs(1)
        
class Car:
    # static properties
    population = 0
    cars = []
    exited = []
    #initializing a car
    def __init__(self, time):
        #store variables on instance
        self.id = Car.population
        self.traveled = 0
        self.history = []
        self.t_arrival = time
        # update the static properties
        Car.population += 1
        Car.cars.append(self)
        
    #traverse an intersection
    def traverse(self,queue):
        self.history.append(queue.id)#record where it's been for debugging
        self.traveled += 1 #increment the travel number
    
    #record the exit time and add car to list of exited vehicles
    def exit(self, time):
        self.t_exit = time
        Car.exited.append(self)

    def __repr__(self):
        return str(vars(self))

queues = map(lambda x: Queue(), range(N))
time = 0
green = False
## function that runs the simulation one time unit
def tick():
    global time, green
    time += 1

    if time % green_time == 0: #this works because our green time is half our cycle length
        green = not green #switch the signal

    if green:
        switches = [] #a list of which cars to move where

        for (i, queue) in enumerate(queues):
            if len(queue) > 0:#if there are cars in the queue
                car = queue.pop() #pop the rightmost car off the queue
                car.traverse(queue) #have the car traverse the queue
                if car.traveled < trip_length: #if it still has somewhere to go
                    receiving_queue = queues[i+1] if i < (N-1) else queues[0] #find the next queue
                    switches.append((car, receiving_queue)) 
                else: #remove the car
                    car.exit(time)

        for (car, receiving_queue) in switches: #add traversed cars to the new queues
            receiving_queue.appendleft(car)
            
    for queue in queues: 
        queue.record(time) #keep a record of the queue length
        queue.tick(time)
            
for i in range(10000):
    tick()

filtered = filter(lambda x: x.t_arrival > 2000,Car.exited)
delays = map(lambda x: x.t_exit-x.t_arrival,filtered)
delays.sort()
print(len(delays)+2000)
print(np.mean(delays))
plt.cla()
sns.distplot(delays)
plt.show()